In [22]:
import pandas as pd
import numpy
import json
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
data_root = Path("data")

In [28]:
rows = []

for year_dir in data_root.iterdir():
    
    # gets rid of .DS_Store
    if year_dir.is_dir():
        year = year_dir.name
        
        # iterate through each year directory
        for json_path in year_dir.glob("*.json"):
            with open(json_path, "r") as f:
                obj = json.load(f)

            if isinstance(obj, list):
                records = obj
            else:
                records = [obj]

            for rec in records:
                abstract_raw = rec.get("awd_abstract_narration") or ""
                abstract_clean = abstract_raw.replace("\r\n", " ")
                rows.append(
                {
                    "year": year,
                    "file": json_path.name,
                    "awd_id": rec.get("awd_id"),
                    "awd_eff_date": rec.get("awd_eff_date"),
                    "awd_amount": rec.get("awd_amount"),
                    "org_dir_long_name": rec.get("org_dir_long_name"),
                    "abstract": abstract_clean
                }
            )

df = pd.DataFrame(rows)

In [29]:
df["abstract"]

0         Understanding how individual scientists intera...
1         The National Research Council (NRC) of the Nat...
2         Anthropogenic carbon dioxide release is causin...
3         The proposal requests Phase III funding for th...
4         INTELLECTUAL MERIT A central question in eukar...
                                ...                        
244839    This project will investigate how interacting ...
244840    General Summary  A great deal of research in p...
244841    ABSTRACT    Non-technical: Elastomeric protein...
244842    The award will support the PI's research on ma...
244843    Technical Abstract  This DMR project creates a...
Name: abstract, Length: 244844, dtype: object